In [ ]:
!pip install langgraph
!pip install langchain-openai
# !pip install openrouter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.80
    Uninstalling langchain-core-0.3.80:
      Successfully uninstalled langchain-core-0.3.80
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.1.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 2.8 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
import numpy as np

In [ ]:
#load dataset qm9
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 21.3 MB/s eta 0:00:00


# Task
Fine-tune a T5 model on the QM9 dataset. This involves loading the dataset, preprocessing it for T5 tokenization, defining training arguments, and then fine-tuning the model using `transformers.Trainer`. Finally, summarize the fine-tuning process and its results.

## Load QM9 Dataset

### Subtask:
Load the QM9 dataset. This might involve downloading it if not already available and parsing it into a suitable format (e.g., pandas DataFrame or Hugging Face Dataset).


**Reasoning**:
The subtask is to load the QM9 dataset. This step imports the necessary function, loads the dataset, and prints its structure for inspection.



In [ ]:
!pip install torch-geometric rdkit



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.6 MB/s eta 0:00:00


In [ ]:
  from torch_geometric.datasets import QM9

dataset = QM9("./qm9")
subset = dataset[:1000]   # ambil 1500 row


Extracting qm9/raw/qm9.zip
Processing...
100%|██████████| 133885/133885 [02:57<00:00, 754.98it/s] 
Done!


In [ ]:
samples = []

for data in subset:
    y = data.y.view(-1)

    mu     = y[0].item()
    alpha  = y[1].item()
    homo   = y[2].item()
    lumo   = y[3].item()
    gap    = lumo - homo
    Cv     = y[11].item()
    num_atoms = data.x.size(0)

    smiles = data.smiles

    caption = (
    f"properties: mu={mu:.4f}, "
    f"alpha={alpha:.4f}, "
    f"gap={gap:.4f}, "
    f"Cv={Cv:.4f}, "
    f"num_atoms={num_atoms}"
)

# caption = "properties: mu=0.3210, alpha=9.2300, gap=1.8123, Cv=20.1410, num_atoms=12"


    samples.append({
        "input": caption,
        "target": smiles
    })


In [ ]:
samples

[{'input': 'properties: mu=0.0000, alpha=13.2100, gap=13.7363, Cv=6.4690, num_atoms=5',
  'target': '[H]C([H])([H])[H]'},
 {'input': 'properties: mu=1.6256, alpha=9.4600, gap=9.2492, Cv=6.3160, num_atoms=4',
  'target': '[H]N([H])[H]'},
 {'input': 'properties: mu=1.8511, alpha=6.3100, gap=9.8369, Cv=6.0020, num_atoms=3',
  'target': '[H]O[H]'},
 {'input': 'properties: mu=0.0000, alpha=16.2800, gap=9.1185, Cv=8.5740, num_atoms=4',
  'target': '[H]C#C[H]'},
 {'input': 'properties: mu=2.8937, alpha=12.9900, gap=10.3267, Cv=6.2780, num_atoms=3',
  'target': '[H]C#N'},
 {'input': 'properties: mu=2.1089, alpha=14.1800, gap=6.1607, Cv=6.4130, num_atoms=4',
  'target': '[H]C([H])=O'},
 {'input': 'properties: mu=0.0000, alpha=23.9500, gap=12.0438, Cv=10.0980, num_atoms=8',
  'target': '[H]C([H])([H])C([H])([H])[H]'},
 {'input': 'properties: mu=1.5258, alpha=16.9700, gap=9.3526, Cv=8.7510, num_atoms=6',
  'target': '[H]OC([H])([H])[H]'},
 {'input': 'properties: mu=0.7156, alpha=28.7800, gap=8.76

In [ ]:
import json

with open("qm9_caption2smiles.jsonl", "w") as f:
    for row in samples:
        f.write(json.dumps(row) + "\n")


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained(
    "laituan245/molt5-base-caption2smiles",
    model_max_length=512
)

model = T5ForConditionalGeneration.from_pretrained(
    "laituan245/molt5-base-caption2smiles"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="qm9_caption2smiles.jsonl")
train = dataset["train"]


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
train[0]


{'input': 'properties: mu=0.0000, alpha=13.2100, gap=13.7363, Cv=6.4690, num_atoms=5',
 'target': '[H]C([H])([H])[H]'}

In [ ]:

def tokenize_fn(batch):
    model_inputs = tokenizer(
        batch["input"],
        padding="max_length",
        truncation=True,
        max_length=128,
    )

    # Tokenize SMILES (target)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["target"],
            padding="max_length",
            truncation=True,
            max_length=128,
        )["input_ids"]

    # PAD → -100
    labels = [
        [-100 if token == tokenizer.pad_token_id else token for token in seq]
        for seq in labels
    ]

    model_inputs["labels"] = labels
    return model_inputs

tokenized = train.map(tokenize_fn, batched=True, remove_columns=train.column_names)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
from transformers import Trainer, TrainingArguments


In [ ]:
training_args = TrainingArguments(
    output_dir="molT5-finetuned",
    per_device_train_batch_size=4,
    learning_rate=2e-4,
    num_train_epochs=5,
    logging_steps=50,
    save_steps=200,
    report_to="none",
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
)
trainer.train()


Step,Training Loss
50,2.486100
100,0.687900
150,0.524900
200,0.470100
250,0.422500
300,0.414400
350,0.367600
400,0.356400
450,0.366200
500,0.324800


TrainOutput(global_step=1250, training_loss=0.4418441879272461, metrics={'train_runtime': 1156.8085, 'train_samples_per_second': 4.322, 'train_steps_per_second': 1.081, 'total_flos': 855946690560000.0, 'train_loss': 0.4418441879272461, 'epoch': 5.0})

In [ ]:
trainer.save_model("./molT5_finetuned_fixed")
tokenizer.save_pretrained("./molT5_finetuned_fixed")


In [ ]:
trainer.save_model("./molT5_finetuned_fixed")
tokenizer.save_pretrained("./molT5_finetuned_fixed")
